In [1]:
import os
import requests
from bs4 import BeautifulSoup

In [3]:
seznam_url = "https://gcatholic.org/documents/data/type-PBH"

In [5]:
seznam_stranka = requests.get(seznam_url)

In [7]:
seznam_polivka = BeautifulSoup(seznam_stranka.text)

In [9]:
dokumenty = []
for radek in seznam_polivka.find_all('tr'):
    if 'Pope' in radek.text:
        dokument = {}
        if radek.find(class_='doc'):
            dokument['nazev'] = radek.find(class_='doc').text.strip()
            dokument['dokument'] = 'bula'
            tagy = []
            for t in radek.find_all(class_='tag'):
                tagy.append(t.text.strip())
            dokument['tagy'] = tagy
            dokument['papez'] = radek.find_all('td')[1].text.strip()
            datum = radek.find_all('td')[2].text.strip()
            if datum == '1831.03':
                datum = '1831.03.01'
            dokument['datum'] = datum
            dokumenty.append(dokument)

In [11]:
import polars as pl

In [13]:
df = pl.DataFrame(dokumenty).with_columns(pl.col('datum').str.to_date())

In [15]:
df.write_parquet(os.path.join('data','buly.parquet'), use_pyarrow=True)